In [ ]:
import re

import rushd as rd
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
sns.set_context('notebook',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
def dox_string_to_float(s):
    # Handle '0'
    if s == '0':
        return 0.0
    # Handle E-1.5 and friends
    if s.startswith('E'):
        return 10**float(s[1:])
    # Handle BASEeEXP (1e-1.5)
    e_split = s.lower().split('e')
    if len(e_split) == 2:
        return float(e_split[0]) * (10**float(e_split[1]))
    raise RuntimeError(f"Unable to convert dox string: '{s}' to float.")
no_yellow_viridis = matplotlib.colors.ListedColormap(matplotlib.cm.get_cmap('viridis', 256)(np.linspace(0,0.85,256)))

In [ ]:
regex = re.compile(r'^export_293T\.(?P<condition>\d-gene(?:\.\w+)?)\.(?P<dox>.*)-dox\.(?P<rep>\d)-rep_Single Cells')
single_gene_df = []
for file in (rd.datadir/'instruments'/'data'/'attune'/'chris'/'2021.05.07-T2.0-transfections'/'csv_export').glob('*.csv'):
    match = regex.match(file.name)
    if match is not None:
        df = pd.read_csv(rd.infile(file))
        df['condition'] = match['condition']
        df['dox'] = dox_string_to_float(match['dox'])
        df['replicate'] = int(match['rep'])
        single_gene_df.append(df)
single_gene_df = pd.concat(single_gene_df, ignore_index=True)
single_gene_df = single_gene_df[(single_gene_df['eGFP-A'] > 0) & (single_gene_df['iRFP670-A'] > 3000) & (single_gene_df.dox > 0)]

In [ ]:
no_yellow_viridis(cm_norm(0.03))

In [ ]:
[*single_gene_df.dox.unique()]

In [ ]:
cm_norm = matplotlib.colors.LogNorm(vmin=0.0003, vmax=0.316)
g = sns.displot(data=single_gene_df, x='eGFP-A', kind='kde', hue='dox', palette=no_yellow_viridis, hue_norm=cm_norm, legend=None, log_scale=True)
for i, dox in enumerate(sorted(single_gene_df.dox.unique())):
    plt.text(x=0.9,y=(17-i)/20, s=f'{dox:.2e}', ha='right', fontsize=12, color=no_yellow_viridis(cm_norm(dox)), transform=plt.gca().transAxes)
plt.text(x=0.7, y=18/20, s='Induction', transform=plt.gca().transAxes)
plt.title('Single-gene cassettes; iRFP noise reporter gated')
plt.savefig(rd.outfile(rd.rootdir/'img'/'single_gene_tet_induction.pdf'), bbox_inches='tight')
plt.savefig(rd.outfile(rd.rootdir/'img'/'single_gene_tet_induction.svg'), bbox_inches='tight')
plt.savefig(rd.outfile(rd.rootdir/'img'/'single_gene_tet_induction.png'), bbox_inches='tight')
plt.show()